In [ ]:
import os                       # 导入os模块，用于与操作系统进行交互，如文件路径操作、目录创建等。
import time                     # 导入time模块，用于时间相关的操作，这里主要用于计时。
import cv2                      # 导入opencv-python库，通常是OpenCV的简称，用于图像处理和计算机视觉任务。
import numpy as np              # 导入numpy库，用于进行高效的数值计算，特别是处理数组和矩阵。
import vision.utils.box_utils_numpy as box_utils  # 导入自定义的box_utils模块（可能是项目内部的），其中包含了边界框相关的工具函数，如NMS。
import onnxruntime as ort       # 导入onnxruntime库并别名为ort，它是运行ONNX模型的引擎，可以高效地执行深度学习模型的推理。

# 定义预测函数，对模型输出的边界框和置信度进行后处理
# `predict` 函数接收模型原始输出的边界框和置信度，经过后处理（如NMS）后，返回最终的检测结果。
# `width`, `height`: 原始图像的宽度和高度，用于将归一化后的边界框坐标还原。
# `confidences`: 模型输出的类别置信度。
# `boxes`: 模型输出的边界框坐标（通常是归一化后的）。
# `prob_threshold`: 置信度阈值，低于此阈值的检测结果将被忽略。
# `iou_threshold=0.3`: IoU（Intersection over Union）阈值，用于非极大值抑制（NMS）。
# `top_k=-1`: 在NMS后保留的最多框的数量，-1表示保留所有。
def predict(width, height, confidences, boxes, prob_threshold, iou_threshold=0.3, top_k=-1):
    boxes = boxes[0]
    # 模型输出通常是批量处理的结果，即使只输入一张图片，输出也会有一个批次维度。
    # `boxes[0]` 获取批次中第一个（也是唯一一个）图像的边界框数据。
    confidences = confidences[0]
    # 同样地，`confidences[0]` 获取批次中第一个图像的类别置信度。

    picked_box_probs = []
    # 初始化一个空列表，用于存储经过后处理（NMS等）后筛选出的边界框及其对应的置信度。
    picked_labels = []
    # 初始化一个空列表，用于存储筛选出的边界框对应的类别标签。

    # 遍历每个类别（通常从类别索引1开始，因为0通常是背景类）
    for class_index in range(1, confidences.shape[1]):
        probs = confidences[:, class_index]
        # 获取当前类别 `class_index` 的所有边界框的置信度分数。
        mask = probs > prob_threshold
        # 创建一个布尔掩码，标记出置信度高于 `prob_threshold` 的边界框。
        probs = probs[mask]
        # 使用掩码筛选出置信度高于阈值的边界框的置信度。
        if probs.shape[0] == 0:
            # 如果当前类别没有任何边界框的置信度超过阈值，则跳过该类别。
            continue
        subset_boxes = boxes[mask, :]
        # 使用相同的掩码筛选出置信度高于阈值的边界框坐标。
        box_probs = np.concatenate([subset_boxes, probs.reshape(-1, 1)], axis=1)
        # 将筛选出的边界框坐标和其对应的置信度拼接成一个数组。
        # `probs.reshape(-1, 1)` 将一维的置信度数组转换为列向量，以便与边界框坐标拼接。
        # 结果是一个 N x 5 的数组，其中 N 是通过阈值筛选后的框的数量，每行是 [x1, y1, x2, y2, confidence]。

        box_probs = box_utils.hard_nms(box_probs,
                                       iou_threshold=iou_threshold,
                                       top_k=top_k,
                                       )
        # 对当前类别的边界框执行非极大值抑制（NMS）。
        # NMS 是一种常用的后处理技术，用于消除重叠的、对同一目标重复检测的边界框，只保留最置信度最高的那个。
        # `iou_threshold` 控制重叠度，`top_k` 控制保留框的数量。

        picked_box_probs.append(box_probs)
        # 将NMS后保留的边界框和置信度添加到 `picked_box_probs` 列表中。
        picked_labels.extend([class_index] * box_probs.shape[0])
        # 将对应类别的标签（`class_index`）添加到 `picked_labels` 列表中，数量与保留的边界框数量一致。

    if not picked_box_probs:
        # 如果 `picked_box_probs` 列表是空的（即没有任何检测结果），则返回空的NumPy数组。
        return np.array([]), np.array([]), np.array([])
    
    picked_box_probs = np.concatenate(picked_box_probs)
    # 将所有类别的、NMS后的边界框和置信度拼接成一个大的NumPy数组。

    # 将归一化后的边界框坐标还原到原始图像尺寸上。
    picked_box_probs[:, 0] *= width   # x1 坐标乘以原始图像宽度
    picked_box_probs[:, 1] *= height  # y1 坐标乘以原始图像高度
    picked_box_probs[:, 2] *= width   # x2 坐标乘以原始图像宽度
    picked_box_probs[:, 3] *= height  # y2 坐标乘以原始图像高度

    return picked_box_probs[:, :4].astype(np.int32), np.array(picked_labels), picked_box_probs[:, 4]
    # 返回最终的检测结果：
    # `picked_box_probs[:, :4].astype(np.int32)`: 原始图像上的整数坐标边界框。
    # `np.array(picked_labels)`: 对应的类别标签数组。
    # `picked_box_probs[:, 4]`: 对应的置信度分数数组。

# 从标签文件中读取每一行，并去除行首尾的空白字符，得到类别名称列表 2分
class_names = [name.strip() for name in open('voc-model-labels.txt').readlines()]
# 打开名为 'voc-model-labels.txt' 的文本文件，逐行读取内容。
# `name.strip()` 去除每行字符串开头和结尾的空白字符（包括换行符）。
# 结果 `class_names` 是一个Python列表，其中每个元素是一个字符串形式的类别名称（例如，'face', 'person'等）。

# 创建 ONNX Runtime 的推理会话，用于运行模型进行推理 2分
ort_session = ort.InferenceSession('version-RFB-320.onnx')
# 使用 `onnxruntime.InferenceSession` 类加载名为 'version-RFB-320.onnx' 的ONNX模型。
# 这个模型通常是一个预训练的目标检测模型，例如用于人脸检测或其他通用目标检测。
# `ort_session` 对象是与加载模型交互的接口，允许我们执行模型的推理。

# 获取模型输入的名称 2分
input_name = ort_session.get_inputs()[0].name
# 从 `ort_session` 对象中获取模型第一个输入节点的名称。
# 目标检测模型通常有一个输入，即图像数据。这个名称在准备输入字典时需要用到。

# 定义保存检测结果图像的目录路径
result_path = "./detect_imgs_results_onnx"
# 定义一个字符串变量 `result_path`，指定了用于保存检测结果图像的目录的相对路径。

# 定义置信度阈值，用于筛选出置信度较高的检测结果
threshold = 0.7
# 设置一个浮点数 `threshold`，表示检测结果的置信度阈值。
# 只有当模型预测的某个目标置信度高于这个值时，才会被认为是有效的检测结果。

# 定义存储待检测图像的目录路径
path = "imgs"
# 定义一个字符串变量 `path`，指定了包含待检测图像的目录的相对路径。

# 用于统计所有图像中检测到的目标框总数，初始化为 0
sum = 0
# 初始化一个整数变量 `sum` 为0，用于累积统计在所有处理的图像中检测到的目标框的总数量。

# 如果保存结果的目录不存在，则创建该目录 2分
if not os.path.exists(result_path):
    # 使用 `os.path.exists()` 检查 `result_path` 指定的目录是否存在。
    os.mkdir(result_path)
    # 如果目录不存在，使用 `os.mkdir()` 创建该目录。

# 获取指定目录下的所有文件和文件夹名称列表
listdir = os.listdir(path)
# 使用 `os.listdir(path)` 获取 `path` 目录下所有文件和子目录的名称列表。

# 遍历目录下的每个文件
for file_path in listdir:
    # 拼接图像文件的完整路径
    img_path = os.path.join(path, file_path)
    # 使用 `os.path.join()` 将目录路径 `path` 和文件名 `file_path` 组合成一个完整的图像文件路径，确保跨平台兼容性。

    # 使用 OpenCV 读取图像文件 2分
    orig_image = cv2.imread(img_path)
    # 使用OpenCV的 `cv2.imread()` 函数读取指定路径的图像文件。
    # `orig_image` 变量将存储原始图像的NumPy数组（默认是BGR格式）。

    # 将图像从 BGR 颜色空间转换为 RGB 颜色空间（许多模型要求输入为 RGB 格式）
    image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
    # OpenCV 默认读取的图像是BGR（蓝绿红）格式。
    # 大多数深度学习模型（尤其是那些在ImageNet上预训练的）期望RGB（红绿蓝）格式的输入。
    # `cv2.cvtColor()` 函数用于进行颜色空间转换。

    # 将图像调整为 320x240 的尺寸（符合模型输入的尺寸要求） 2分
    image = cv2.resize(image, (320, 240))
    # 使用 `cv2.resize()` 函数将图像调整为指定的宽度 `320` 和高度 `240` 像素。
    # 目标检测模型通常有固定的输入尺寸要求，这里根据模型需要进行尺寸调整。

    # 定义图像归一化的均值数组 2分
    image_mean = np.array([127, 127, 127])
    # 定义一个NumPy数组 `image_mean`，包含RGB三个通道的像素均值。
    # 这些值通常是模型训练时使用的归一化参数。

    # 对图像进行归一化处理，减去均值并除以 128
    image = (image - image_mean) / 128
    # 对图像像素值进行归一化和标准化处理。
    # 首先，从每个像素值中减去对应的通道均值 `image_mean`。
    # 然后，将结果除以 128 （通常是标准差或某个固定值），将像素值约束在一个特定范围（例如[-1, 1]），以适应模型输入层的要求。

    # 将图像的维度从 (高度, 宽度, 通道数) 转换为 (通道数, 高度, 宽度)
    image = np.transpose(image, [2, 0, 1])
    # 原始图像数组的维度顺序通常是 (H, W, C)（高、宽、通道）。
    # 大多数深度学习模型（特别是PyTorch或某些ONNX模型）期望的输入维度顺序是 (C, H, W)（通道、高、宽）。
    # `np.transpose(image, [2, 0, 1])` 将维度从 [H, W, C] 调整为 [C, H, W]。

    # 在第一个维度上扩展一个维度，将图像变为 (1, 通道数, 高度, 宽度)，以符合模型输入的维度要求  1分
    image = np.expand_dims(image, axis=0)
    # 使用 `np.expand_dims()` 在数组的最前面（轴0）添加一个新维度。
    # 模型通常期望接收一个批次（Batch）的图像作为输入，即使只有一张图片，也需要将其封装在批次维度中。
    # 此时图像的形状变为 (1, C, H, W)，其中 `1` 表示批次大小为1。

    # 将图像数据类型转换为 float32 类型
    image = image.astype(np.float32)
    # 将NumPy数组 `image` 的数据类型转换为 `np.float32`。
    # 这是深度学习模型通常要求的输入数据类型，有助于模型内部的计算精度和兼容性。

    # 记录开始时间，用于计算模型推理的耗时
    time_time = time.time()
    # 使用 `time.time()` 获取当前时间戳，标记推理开始的时间。

    # 使用 ONNX Runtime 运行模型，输入图像数据，得到模型输出的置信度和边界框  2分
    confidences, boxes = ort_session.run(None, {input_name: image})
    # 使用 `ort_session.run()` 方法执行模型的推理（即目标检测）。
    # 第一个参数 `None` 表示我们希望获取模型所有的输出。
    # 第二个参数 `{input_name: image}` 是一个字典，键是模型输入节点的名称，值是准备好的图像数据。
    # 模型会返回两个主要输出：`confidences`（每个类别的置信度分数）和 `boxes`（检测到的边界框坐标）。

    # 计算并打印模型推理的耗时
    print("cost time:{}".format(time.time() - time_time))
    # 再次获取当前时间戳，计算与 `time_time` 的差值，得到模型推理所花费的时间，并打印出来。

    # 调用 predict 函数对模型输出的边界框和置信度进行后处理，得到最终的边界框、类别标签和置信度
    boxes, labels, probs = predict(orig_image.shape[1], orig_image.shape[0], confidences, boxes, threshold)
    # 调用自定义的 `predict` 函数，将模型原始输出的 `confidences` 和 `boxes` 进行后处理。
    # `orig_image.shape[1]` 是原始图像的宽度，`orig_image.shape[0]` 是原始图像的高度，用于坐标还原。
    # `threshold` 是预设的置信度阈值。
    # 函数返回处理后的实际边界框坐标 `boxes`、对应的类别标签 `labels` 和最终的置信度 `probs`。

    # 遍历每个检测到的目标框
    for i in range(boxes.shape[0]):
        # 获取当前目标框的坐标
        box = boxes[i, :]
        # `box` 是一个包含 `[x1, y1, x2, y2]` 坐标的NumPy数组。

        # 生成当前目标框的标签字符串，包含类别名称和置信度
        label = f"{class_names[labels[i]]}: {probs[i]:.2f}"
        # 使用f-string格式化生成一个标签字符串，例如 "face: 0.98"。
        # `class_names[labels[i]]` 通过索引获取类别名称。
        # `{probs[i]:.2f}` 格式化置信度，保留两位小数。

        # 在原始图像上绘制目标框，颜色为 (255, 255, 0)，线条粗细为 4
        cv2.rectangle(orig_image, (box[0], box[1]), (box[2], box[3]), (255, 255, 0), 4)
        # 使用OpenCV的 `cv2.rectangle()` 函数在 `orig_image` 上绘制一个矩形框。
        # `(box[0], box[1])` 是矩形的左上角坐标。
        # `(box[2], box[3])` 是矩形的右下角坐标。
        # `(255, 255, 0)` 定义了矩形框的颜色（BGR格式的黄色）。
        # `4` 定义了矩形框的线条粗细。

        # 在图像上绘制类别标签和置信度文本
        cv2.putText(orig_image, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        # 使用 `cv2.putText()` 函数在图像上绘制文本标签。
        # `(box[0], box[1] - 10)` 是文本的起始坐标，稍微向上偏移，避免与框重叠。
        # `cv2.FONT_HERSHEY_SIMPLEX` 指定字体。
        # `0.5` 指定字体大小。
        # `(0, 255, 255)` 定义文本颜色（BGR格式的黄色）。
        # `2` 定义文本线条粗细。

        # 将绘制了目标框的图像保存到结果目录中
        cv2.imwrite(os.path.join(result_path, file_path), orig_image)
        # 使用 `cv2.imwrite()` 将带有检测结果的图像保存到 `result_path` 目录中，文件名与原始文件相同。

    # 累加当前图像中检测到的目标框数量到总数中
    sum += boxes.shape[0]
    # `boxes.shape[0]` 表示当前图像中检测到的目标框的数量。
    # 将这个数量加到 `sum` 变量中，用于统计所有图像中检测到的目标框总数。

# 打印所有图像中检测到的目标框总数
print("sum:{}".format(sum))
# 循环结束后，打印出所有检测到的目标框的总数量。
